<a href="https://colab.research.google.com/github/EEG-Easy/EEG-Machinelearning/blob/master/2020-2Project/2_%EB%94%A5%EB%9F%AC%EB%8B%9D/finalP300modelcode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from __future__ import absolute_import, division, print_function, unicode_literals
import functools

import tensorflow as tf
tf.enable_eager_execution()
from tensorflow import keras
import os

import numpy as np
import pandas as pd

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

tf.keras.backend.clear_session()

In [0]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
cd /content/gdrive/My Drive/eeg

/content/gdrive/My Drive/eeg


In [0]:
LABEL_COLUMN = 'group'
LABELS = [0, 1]

CSV_COLUMNS = ['condition','group', 'F1', 'FC3', 'FC1', 'AFz', 'Fz', 'F2', 'FCz']

def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5, # Artificially small to make examples easier to show.
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  return dataset

temp_dataset = get_dataset('RobustP300.csv', select_columns=CSV_COLUMNS)

Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.


In [0]:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

In [0]:
NUMERIC_FEATURES = ['F1', 'FC3', 'FC1', 'AFz', 'Fz', 'F2', 'FCz']

packed_data = temp_dataset.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [0]:
import pandas as pd
desc = pd.read_csv('RobustP300.csv')[NUMERIC_FEATURES].describe()
desc

,F1,FC3,FC1,AFz,Fz,F2,FCz
count,23201.000000,23201.000000,23201.000000,23201.000000,23201.000000,23201.000000,23201.000000
mean,0.169534,0.147238,0.125680,0.257439,0.161898,0.181233,0.115794
std,1.137716,1.068238,1.004288,1.523621,1.147731,1.186636,0.957781
min,-8.638409,-7.128760,-6.194677,-10.138734,-7.300168,-7.736735,-6.063609
25%,-0.450238,-0.453220,-0.460870,-0.440929,-0.455397,-0.448035,-0.462869
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.549762,0.546780,0.539130,0.559071,0.544603,0.551965,0.537131
max,17.795594,20.475962,24.351948,25.661585,17.733415,16.691511,13.711209


In [0]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [0]:
def normalize_numeric_data(data, mean, std):
  # Center the data
  return (data-mean)/std

In [0]:
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(7,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x7fa4d47f56a8>, mean=array([0.16953369, 0.14723849, 0.1256803 , 0.25743903, 0.16189793,
       0.18123256, 0.11579391]), std=array([1.13771572, 1.06823785, 1.00428802, 1.52362056, 1.14773099,
       1.18663606, 0.95778078])))

In [0]:
example_batch, labels_batch = next(iter(packed_data)) 
example_batch['numeric']

<tf.Tensor: id=99, shape=(5, 7), dtype=float32, numpy=
array([[ 0.56997234,  0.39992845,  0.88199115,  1.2563653 ,  0.865245  ,
         0.3329931 ,  0.49335223],
       [ 0.2240353 ,  0.41365886,  0.40247983, -0.36064705,  0.06743959,
         0.12470734,  0.3182446 ],
       [-0.34354183,  0.6368739 , -0.7454713 , -0.80029356, -0.9084039 ,
         0.27992165, -1.0512424 ],
       [-0.23497479, -0.39232722, -0.19467014, -0.2312169 , -0.28695983,
        -0.28301573, -0.089685  ],
       [-0.88699013, -1.029036  , -0.41980216, -0.6869951 , -0.8023821 ,
        -0.8498319 , -0.7646797 ]], dtype=float32)>

In [0]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[ 0.35196725,  0.2365484 ,  0.75308156,  0.65562665,  0.6128153 ,
         0.12789139,  0.3942012 ],
       [ 0.04790442,  0.24940172,  0.27561766, -0.4056693 , -0.08230007,
        -0.04763484,  0.21137477],
       [-0.4509699 ,  0.45835802, -0.86743194, -0.69422305, -0.9325372 ,
         0.08316711, -1.2184795 ],
       [-0.35554442, -0.5050988 , -0.31898263, -0.32072023, -0.39108273,
        -0.39123055, -0.21453647],
       [-0.928636  , -1.1011354 , -0.54315335, -0.6198617 , -0.8401621 ,
        -0.8688969 , -0.91928506]], dtype=float32)

In [0]:
CATEGORIES = {'condition': [1, 2, 3]}

In [0]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [0]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
[0. 0. 1.]


In [0]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns) #+numeric_columns

In [0]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 체크포인트 콜백 만들기
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,

    period=100)

In [0]:
model = tf.keras.Sequential([
  preprocessing_layer, # 256,256 / 128,256
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(256, activation='relu'),
  #tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(
    loss='binary_crossentropy', optimizer='adam',
    metrics=['accuracy']) 

In [0]:
#first 300 epochs with data 20000
train_data = packed_data.shuffle(20000)
test_data = packed_data.shuffle(1000)

In [0]:
model.fit(train_data, epochs=300, callbacks = [cp_callback])

Epoch 1/300
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
4641/4641 [==============================] - 24s 5ms/step - loss: 0.6702 - acc: 0.6012
Epoch 2/300
4641/4641 [==============================] - 11s 2ms/step - loss: 0.6632 - acc: 0.6084
Epoch 3/300
4641/4641 [==============================] - 11s 2ms/step - loss: 0.6599 - acc: 0.6127
Epoch 4/300
4641/4641 [==============================] - 11s 2ms/step - loss: 0.6566 - acc: 0.6161
Epoch 5/300
4641/4641 [==============================] - 11s 2ms/step - loss: 0.6542 - acc: 0.6167
Epoch 6/300
4641/4641 [==============================] - 11s 2ms/step - loss: 0.6515 - acc: 0.6190
Epoch 7/300
4641/4641 [==============================] - 11s 2ms/step - loss: 0.6490 - acc: 0.6206
Epoch 8/300
4641/4641 [==============================] - 11s 2ms/step - loss: 0.6472 - acc: 0.6216
Epoch 9/300
4641/4641 [==============================] - 11s 2ms/step - loss: 0.6454 - acc: 0.6243
Epoch 10/300
46

In [0]:
test_loss, test_accuracy = model.evaluate(test_data, verbose=2)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

4641/4641 - 6s - loss: 0.5293 - acc: 0.7832


Test Loss 0.5292632074737235, Test Accuracy 0.7831990122795105


In [0]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
model.load_weights(latest)

In [0]:
checkpoint_path = "training_2/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 체크포인트 콜백 만들기
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,

    period=100)

In [0]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
model.load_weights(latest)

In [0]:
#second 300 epochs with 10000 data
train_data = packed_data.shuffle(10000)
test_data = packed_data.shuffle(1000)

In [0]:
model.fit(train_data, epochs=300, callbacks = [cp_callback])

Epoch 1/300
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
4641/4641 [==============================] - 29s 6ms/step - loss: 0.3065 - acc: 0.8678
Epoch 2/300
4641/4641 [==============================] - 12s 3ms/step - loss: 0.2809 - acc: 0.8738
Epoch 3/300
4641/4641 [==============================] - 12s 2ms/step - loss: 0.2670 - acc: 0.8749
Epoch 4/300
4641/4641 [==============================] - 12s 3ms/step - loss: 0.2758 - acc: 0.8741
Epoch 5/300
4641/4641 [==============================] - 11s 2ms/step - loss: 0.2685 - acc: 0.8792
Epoch 6/300
4641/4641 [==============================] - 11s 2ms/step - loss: 0.2749 - acc: 0.8759
Epoch 7/300
4641/4641 [==============================] - 11s 2ms/step - loss: 0.2658 - acc: 0.8788
Epoch 8/300
4641/4641 [==============================] - 12s 3ms/step - loss: 0.2596 - acc: 0.8816
Epoch 9/300
4641/4641 [==============================] - 12s 3ms/step - loss: 0.2785 - acc: 0.8762
Epoch 10/300
46

In [0]:
test_loss, test_accuracy = model.evaluate(test_data, verbose=2)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

4641/4641 - 6s - loss: 0.2935 - acc: 0.8853


Test Loss 0.2934998929088899, Test Accuracy 0.8853497505187988


In [0]:
checkpoint_path = "training_3/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 체크포인트 콜백 만들기
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,

    period=100)

In [0]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
model.load_weights(latest)

In [0]:
#third 300 epochs x3
train_data = packed_data.shuffle(10000)
test_data = packed_data.shuffle(1000)

In [0]:
model.fit(train_data, epochs=300, callbacks = [cp_callback])

Epoch 1/300
4641/4641 [==============================] - 28s 6ms/step - loss: 0.2080 - acc: 0.9238
Epoch 2/300
4641/4641 [==============================] - 12s 3ms/step - loss: 0.1785 - acc: 0.9294
Epoch 3/300
4641/4641 [==============================] - 12s 3ms/step - loss: 0.1799 - acc: 0.9283
Epoch 4/300
4641/4641 [==============================] - 12s 3ms/step - loss: 0.2064 - acc: 0.9256
Epoch 5/300
4641/4641 [==============================] - 12s 3ms/step - loss: 0.1551 - acc: 0.9325
Epoch 6/300
4641/4641 [==============================] - 12s 3ms/step - loss: 0.1702 - acc: 0.9277
Epoch 7/300
4641/4641 [==============================] - 12s 3ms/step - loss: 0.1529 - acc: 0.9314
Epoch 8/300
4641/4641 [==============================] - 12s 3ms/step - loss: 0.1741 - acc: 0.9297
Epoch 9/300
4641/4641 [==============================] - 12s 3ms/step - loss: 0.1600 - acc: 0.9300
Epoch 10/300
4641/4641 [==============================] - 12s 3ms/step - loss: 0.1770 - acc: 0.9283
Epoch 11/

KeyboardInterrupt: ignored

In [0]:
test_loss, test_accuracy = model.evaluate(test_data, verbose=2)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

4641/4641 - 6s - loss: 0.1758 - acc: 0.9305


Test Loss 0.17581541583844792, Test Accuracy 0.9304771423339844


In [0]:
checkpoint_path = "training_4/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 체크포인트 콜백 만들기
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,

    period=100)

In [0]:
#4th 300 epochs
train_data = packed_data.shuffle(70000)
test_data = packed_data.shuffle(200)

In [0]:
model.fit(train_data, epochs=100, callbacks = [cp_callback])

Epoch 1/100
4641/4641 [==============================] - 28s 6ms/step - loss: 0.1514 - acc: 0.9328
Epoch 2/100
4641/4641 [==============================] - 13s 3ms/step - loss: 0.1647 - acc: 0.9325
Epoch 3/100
4641/4641 [==============================] - 13s 3ms/step - loss: 0.2122 - acc: 0.9267
Epoch 4/100
4641/4641 [==============================] - 12s 3ms/step - loss: 0.1611 - acc: 0.9323
Epoch 5/100
4641/4641 [==============================] - 12s 3ms/step - loss: 0.1723 - acc: 0.9297
Epoch 6/100
4641/4641 [==============================] - 12s 3ms/step - loss: 0.1733 - acc: 0.9311
Epoch 7/100
4641/4641 [==============================] - 12s 3ms/step - loss: 0.1594 - acc: 0.9330
Epoch 8/100
4641/4641 [==============================] - 12s 3ms/step - loss: 0.1606 - acc: 0.9320
Epoch 9/100
4641/4641 [==============================] - 12s 3ms/step - loss: 0.1961 - acc: 0.9285
Epoch 10/100
4641/4641 [==============================] - 13s 3ms/step - loss: 0.1479 - acc: 0.9341
Epoch 11/

In [0]:
test_loss, test_accuracy = model.evaluate(test_data, verbose=2)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

4641/4641 - 6s - loss: 0.1750 - acc: 0.9320


Test Loss 0.17497062047196152, Test Accuracy 0.9319856762886047
